In [14]:
import numpy as np
import pandas as pd
import os
from libs.utils import *
from libs.tf_idf import *

In [6]:
DATASET_PATH = r"E:\Learn Machine Learning\Project\Opinion Classifier\res\dataset\nremove_stopwords.xlsx"

data : pd.DataFrame = read_dataset(DATASET_PATH, "xlsx", header=0)

data.head()

15:21:03 root INFO: Đang đọc dữ liệu từ file E:\Learn Machine Learning\Project\Opinion Classifier\res\dataset\nremove_stopwords.xlsx ...
		at Line 330 [read_dataset() in utils.py, utils]
15:21:04 root INFO: Đọc thành công! 
		at Line 338 [read_dataset() in utils.py, utils]


,repair_words,labels
0,"['tiếng', 'anh', 'thi', 'đầu_vào', 'là', 'khôn...",Neutral
1,"['đã', 'từng', 'là', 'niềm', 'mơ_ước', 'khi', ...",Negative
2,"['học_phí', 'hơi', 'cao']",Negative
3,"['̀', '̀', '̣', '̀', '̀', '̉', '̣', '̂', 'đ', ...",Neutral
4,"['thiết_nghĩ', 'đề', 'thi', 'utc', 'nên', 'đổi...",Positive


In [13]:
labels : list[str] = data["labels"].unique().tolist()
labels.sort(reverse=True)
labels

['Positive', 'Neutral', 'Negative']

In [18]:
import string
import re

string_pattele = string.punctuation + '\n'
string_pattele


def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_punctuation(text, except_char: str = None):
    try:
        result_text = "".join(
            char for char in text if char not in string_pattele or (except_char and char == except_char))
        result_text = remove_emoji(result_text)
    except TypeError:
        result_text = None
    return result_text

def filter_none_text_id(documents : pd.Series | list[str]) -> list[int]:
    need_filter : list[str] = [text for text in documents]
    none_text_index : list[int] = []
    for i, text in enumerate(need_filter):
        if (text is None):
            print(str(i) + ": " + str(text))
            none_text_index.append(i)
    
    return none_text_index

In [20]:
data["Tokens"] = data["repair_words"].apply(lambda text: [remove_punctuation(x, '_').split()[0] for x in text.split(',')])

In [29]:
corpus: list[list[str]] = [doc for doc in data["Tokens"]]
labels_corpus: list[str] = data["labels"].to_list()

In [38]:
vocabulary = features_extraction(corpus)
vocabulary = list(vocabulary)
vocabulary.sort()

Tạo phân phối tần suất:

In [46]:
positive_frequency : dict[str, int] = {}
for token in vocabulary:
    positive_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Positive"].values:
    for token in sen:
        positive_frequency[token] += 1

negative_frequency : dict[str, int] = {}
for token in vocabulary:
    negative_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Negative"].values:
    for token in sen:
        negative_frequency[token] += 1

neutral_frequency : dict[str, int] = {}
for token in vocabulary:
    neutral_frequency[token] = 0
for _, label, sen in data[data["labels"] == "Neutral"].values:
    for token in sen:
        neutral_frequency[token] += 1


ValueError: If using all scalar values, you must pass an index